In [27]:
#libraries

import findspark
import pandas as pd
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StringType, DoubleType, IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import RandomForestClassificationModel
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("FraudDetectionStreaming") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0") \
    .getOrCreate()

25/04/18 14:45:57 WARN Utils: Your hostname, Dulyas-Laptop.local resolves to a loopback address: 127.0.0.1; using 192.168.177.129 instead (on interface en0)
25/04/18 14:45:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/dulyanethmini/.ivy2/cache
The jars for the packages stored in: /Users/dulyanethmini/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a6189060-2b03-469b-8d50-2ed43535ca50;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.0 in central


:: loading settings :: url = jar:file:/Users/dulyanethmini/anaconda3/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.0 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in local-m2-cache
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 216ms :: artifacts dl 5ms
	:: modules in use:
	com.google.code.findbugs#jsr305;3.0.0 from central in [default]
	commons-logging#commons-logging;1.1.3 from local-m2-cache in [default]
	org.apache.commons#commons-pool2;2.11.1 from central in [default]
	org.apache.hadoop#hadoop-client-api;3.3.2 from central in [default]
	org.apache.hadoop

In [5]:
# Define the schema for JSON messages
schema = StructType() \
    .add("type", StringType()) \
    .add("amount", DoubleType()) \
    .add("oldbalanceOrg", DoubleType()) \
    .add("newbalanceOrig", DoubleType()) \
    .add("oldbalanceDest", DoubleType()) \
    .add("newbalanceDest", DoubleType()) \
    .add("isFraud", IntegerType())

In [7]:
# Read data from Kafka
df_kafka = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "fraud-topic") \
    .option("startingOffsets", "latest") \
    .load()


In [9]:
# Parse JSON and extract fields
df_parsed = df_kafka.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data")) \
    .select("data.*")

In [11]:
indexer = StringIndexer(inputCol="type", outputCol="typed_index")

In [13]:
# Fit on a static small DataFrame to simulate the mapping (for streaming use)
types_df = spark.createDataFrame([("CASH_OUT",), ("TRANSFER",), ("DEBIT",), ("CASH_IN",), ("PAYMENT",)], ["type"])
index_model = indexer.fit(types_df)

In [15]:
# Transform streaming DataFrame
df_transformed = index_model.transform(df_parsed)


In [17]:
df_transformed.printSchema()

root
 |-- type: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- oldbalanceOrg: double (nullable = true)
 |-- newbalanceOrig: double (nullable = true)
 |-- oldbalanceDest: double (nullable = true)
 |-- newbalanceDest: double (nullable = true)
 |-- isFraud: integer (nullable = true)
 |-- typed_index: double (nullable = false)



In [19]:
# Load your pre-trained Random Forest model
rf_model = RandomForestClassificationModel.load("rf_model_path")

In [23]:


input_cols = df_transformed.select(
    "typed_index",  
    "amount",
    "oldbalanceOrg",
    "newbalanceOrig",
    "oldbalanceDest",
    "newbalanceDest"
)


In [25]:
# Use the trained model to make predictions
df_predictions = rf_model.transform(input_cols)



IllegalArgumentException: features does not exist. Available: typed_index, amount, oldbalanceOrg, newbalanceOrig, oldbalanceDest, newbalanceDest

In [29]:


# Assemble the feature columns into a single "features" column
assembler = VectorAssembler(
    inputCols=["typed_index", "amount", "oldbalanceOrg", "newbalanceOrig", "oldbalanceDest", "newbalanceDest"],
    outputCol="features"
)

# Transform the data to include the features column
df_transformed = assembler.transform(input_cols)


In [31]:
# Use the trained Random Forest model to make predictions
df_predictions = rf_model.transform(df_transformed)

# Show the predictions
df_predictions.select("features", "prediction").show()


AnalysisException: Queries with streaming sources must be executed with writeStream.start();
kafka

In [33]:
# Use writeStream to output the predictions to the console
query = df_predictions.select("features", "prediction").writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

# Wait for the stream to terminate (i.e., run indefinitely)
query.awaitTermination()


25/04/18 14:49:12 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/44/y4n69c6j4ls1w4dvtlx32jfc0000gn/T/temporary-682a2966-cb52-44a0-acbf-1f98112891b4. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/18 14:49:12 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/04/18 14:49:12 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
25/04/18 14:49:12 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
25/04/18 14:49:12 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
25/04/18 14:49:12 WARN AdminClientConfig: The configuration '

StreamingQueryException: [STREAM_FAILED] Query [id = 672055be-e150-41f3-a8f7-3566eed2eb2b, runId = 98d7577a-fca4-4f10-9f3a-fd6d64a2d56f] terminated with exception: org.apache.kafka.common.errors.TimeoutException: Call(callName=describeTopics, deadlineMs=1744968134401, tries=1, nextAllowedTryMs=1744968134502) timed out at 1744968134402 after 1 attempt(s)

In [ ]:
kafka-console-consumer --bootstrap-server localhost:9092 --topic fraud-topic --from-beginning


In [36]:
# Ensure proper streaming context for Kafka
from pyspark.sql.functions import expr

# Create a streaming DataFrame from Kafka
kafka_df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "fraud-topic") \
    .load()

# Assuming your data is in JSON format, parse it
df_transformed = kafka_df.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data")) \
    .select("data.*")  # Flatten the nested structure

# Your transformations (feature creation, encoding, etc.) should happen here


In [38]:
from pyspark.ml.feature import VectorAssembler

# Define your feature columns
feature_columns = ["amount", "oldbalanceOrg", "newbalanceOrig", "oldbalanceDest", "newbalanceDest"]

# Create the VectorAssembler
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Apply the VectorAssembler to your DataFrame
df_transformed = assembler.transform(df_transformed)
